## 3.7.1.3.python_crawler_3

In [1]:
import requests
import lxml.html

In [2]:
def main():
    """
    크롤러의 메인 처리
    """
    # 여러 페이지에서 크롤링을 위해 Session 사용
    session = requests.Session()
    
    # scrape_list_page() 함수를 호출해서 제너레이터를 추출
    response = session.get('http://www.hanbit.co.kr/store/books/new_book_list.html')
    urls = scrape_list_page(response)
    
    # 제너레이터는 list처럼 사용 가능
    for url in urls:
        print(url)
        print('-'*70)

In [3]:
def scrape_list_page(response):
    root = lxml.html.fromstring(response.content)
    root.make_links_absolute(response.url)
    for a in root.cssselect('.view_box .book_tit a'):
        url = a.get('href')
        # yield 구문으로 제너레이터의 요소 반환
        yield url

In [4]:
if __name__ == '__main__':
    main()

https://www.hanbit.co.kr/store/books/look.php?p_code=B3178834938
----------------------------------------------------------------------
https://www.hanbit.co.kr/store/books/look.php?p_code=B6084607806
----------------------------------------------------------------------
https://www.hanbit.co.kr/store/books/look.php?p_code=B8948111854
----------------------------------------------------------------------
https://www.hanbit.co.kr/store/books/look.php?p_code=B8661740335
----------------------------------------------------------------------
https://www.hanbit.co.kr/store/books/look.php?p_code=B1382037871
----------------------------------------------------------------------
https://www.hanbit.co.kr/store/books/look.php?p_code=B2847674054
----------------------------------------------------------------------
https://www.hanbit.co.kr/store/books/look.php?p_code=B4162156367
----------------------------------------------------------------------
https://www.hanbit.co.kr/store/books/look.php?p_

## 3.7.2.1.python_crawler_4

In [5]:
import requests
import lxml.html

In [6]:
def main():
    # 여러 페이지에서 크롤링을 위해 Session 사용
    session = requests.Session()
    
    # scrape_list_page() 함수를 호출해서 제너레이터를 추출
    response = session.get('http://www.hanbit.co.kr/store/books/new_book_list.html')
    urls = scrape_list_page(response)
    for url in urls:
        response = session.get(url)
        ebook = scrape_detail_page(response)
        print(ebook)
        break

In [7]:
def scrape_list_page(response):
    root = lxml.html.fromstring(response.content)
    root.make_links_absolute(response.url)
    for a in root.cssselect('.view_box .book_tit a'):
        url = a.get('href')
        yield url

In [8]:
def scrape_detail_page(response):
    """
    상세 페이지의 Response에서 책 정보를 dict로 추출
    """
    root = lxml.html.fromstring(response.content)
    ebook = {
        'url': response.url,
        'title': root.cssselect('.store_product_info_box h3')[0].text_content(),
        'price': root.cssselect('.pbr strong')[0].text_content(),
        'content': [p.text_content()\
                   for p in root.cssselect('#tabs_3 .hanbit_edit_view p')]
    }
    return ebook

In [9]:
if __name__ == '__main__':
    main()

{'url': 'https://www.hanbit.co.kr/store/books/look.php?p_code=B3178834938', 'title': '받침 없는 한글 동화 : 4권 세트', 'price': '49,680', 'content': ['', '\r\n\t\t<1편>\r\n', '이렇게 읽어요', '1. 무시무시 마녀가 이사 와!', '2. 아기 고래 뿌우의 노래', '\xa0', '\r\n\t\t<2편>\r\n', '이렇게 읽어요', '1. 도, 도, 도깨비다!', '2. 느티나무 아파트', '\xa0', '\r\n\t\t<3편>\r\n', '이렇게 읽어요', '1. 바쁘다 바빠 너구리 바빠', '2. 꼬마 지우개 마구마구', '\xa0', '\r\n\t\t<4편>\r\n', '이렇게 읽어요', '1. 도깨비 파자마 파티', '2. 꼬마 두루미 뚜루']}


## 3.7.2.2.python_crawler_5

In [10]:
import requests
import lxml.html
import re

In [11]:
def main():
    # 여러 페이지에서 크롤링을 위해 Session 사용
    session = requests.Session()
    
    # scrape_list_page() 함수를 호출해서 제너레이터를 추출
    response = session.get('http://www.hanbit.co.kr/store/books/new_book_list.html')
    urls = scrape_list_page(response)
    for url in urls:
        response = session.get(url)
        ebook = scrape_detail_page(response)
        print(ebook)
        break

In [12]:
def scrape_list_page(response):
    root = lxml.html.fromstring(response.content)
    root.make_links_absolute(response.url)
    for a in root.cssselect('.view_box .book_tit a'):
        url = a.get('href')
        yield url

In [13]:
def scrape_detail_page(response):
    """
    상세 페이지의 Response에서 책 정보를 dict로 추출
    """
    root = lxml.html.fromstring(response.content)
    ebook = {
        'url': response.url,
        'title': root.cssselect('.store_product_info_box h3')[0].text_content(),
        'price': [normalize_spaces(p.text_content())
                 for p in root.cssselect('#tabs_3 .hanbit_edit_view p')
                 if normalize_spaces(p.text_content()) != '']
    }
    return ebook

In [14]:
def normalize_spaces(s):
    """
    연결된 공백을 하나의 공백으로 변경
    """
    return re.sub(r'\s+', '', s).strip()

In [15]:
if __name__ == '__main__':
    main()

{'url': 'https://www.hanbit.co.kr/store/books/look.php?p_code=B3178834938', 'title': '받침 없는 한글 동화 : 4권 세트', 'price': ['<1편>', '이렇게읽어요', '1.무시무시마녀가이사와!', '2.아기고래뿌우의노래', '<2편>', '이렇게읽어요', '1.도,도,도깨비다!', '2.느티나무아파트', '<3편>', '이렇게읽어요', '1.바쁘다바빠너구리바빠', '2.꼬마지우개마구마구', '<4편>', '이렇게읽어요', '1.도깨비파자마파티', '2.꼬마두루미뚜루']}


## 3.7.4.1.python_crawler_final

In [16]:
import time
import requests
import lxml.html
import re

In [17]:
def main():
    # 여러 페이지에서 크롤링을 위해 Session 사용
    session = requests.Session()
    
    # scrape_list_page() 함수를 호출해서 제너레이터를 추출
    response = session.get('http://www.hanbit.co.kr/store/books/new_book_list.html')
    urls = scrape_list_page(response)
    for url in urls:
        time.sleep(1)
        response = session.get(url)
        ebook = scrape_detail_page(response)
        print(ebook)
        break

In [18]:
def scrape_list_page(response):
    root = lxml.html.fromstring(response.content)
    root.make_links_absolute(response.url)
    for a in root.cssselect('.view_box .book_tit a'):
        url = a.get('href')
        yield url

In [19]:
def scrape_detail_page(response):
    """
    상세 페이지의 Response에서 책 정보를 dict로 추출
    """
    root = lxml.html.fromstring(response.content)
    ebook = {
        'url': response.url,
        'title': root.cssselect('.store_product_info_box h3')[0].text_content(),
        'price': root.cssselect('.pbr strong')[0].text_content(),
        'content': [normalize_spaces(p.text_content())
                   for p in root.cssselect('#tabs_3 .hanbit_edit_view p')
                   if normalize_spaces(p.text_content()) != '']
    }
    return ebook

In [20]:
def normalize_spaces(s):
    """
    연결된 공백을 하나의 공백으로 변경
    """
    return re.sub(r'\s+', '', s).strip()

In [21]:
if __name__ == '__main__':
    main()

{'url': 'https://www.hanbit.co.kr/store/books/look.php?p_code=B3178834938', 'title': '받침 없는 한글 동화 : 4권 세트', 'price': '49,680', 'content': ['<1편>', '이렇게읽어요', '1.무시무시마녀가이사와!', '2.아기고래뿌우의노래', '<2편>', '이렇게읽어요', '1.도,도,도깨비다!', '2.느티나무아파트', '<3편>', '이렇게읽어요', '1.바쁘다바빠너구리바빠', '2.꼬마지우개마구마구', '<4편>', '이렇게읽어요', '1.도깨비파자마파티', '2.꼬마두루미뚜루']}
